# CellRank basics
## start with mouse dt dataset merged adata file (adata+ldata)

## Import packages & data

In [1]:
import scvelo as scv
import scanpy as sc
import cellrank as cr
import numpy as np
import pandas as pd

scv.settings.verbosity = 3
scv.settings.set_figure_params("scvelo")
cr.settings.verbosity = 3

In [2]:
import warnings

warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)

In [ ]:
adata = sc.read_h5ad('../../cell_rank/datasets/mudata_adata_ldata_merged.h5ad')
adata # this anndata doesn't have loom files incorportated.

In [ ]:
#plot umap to check
sc.pl.umap(adata, color='seurat_clusters', frameon=False, title='Endothalial Cell population')
#, save='_clusters.pdf')

In [ ]:
import pandas as pd

In [ ]:
#adata.obs['seurat_clusters'].astype('category')
adata.obs['seurat_clusters']=adata.obs['seurat_clusters'].astype('category')

In [ ]:
scv.pl.proportions(adata, groupby='seurat_clusters')

In [ ]:
scv.pl.proportions(adata, groupby='Sample')

In [ ]:
adata.obs

In [ ]:
# 2022-Oct-12 notes and also checked the notebook online on 2022-11-08 pre-processing
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=3000)#original: 2000
sc.tl.pca(adata)
sc.pp.neighbors(adata, n_pcs=30, n_neighbors=30)
scv.pp.moments(adata, n_pcs=None, n_neighbors=None)

In [ ]:
# 2022-10-12 runned these two lines
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=3000)
scv.pp.moments(adata, n_pcs=40, n_neighbors=30)

In [ ]:
adata

In [ ]:
adata.obs # check obs table

In [ ]:
adata.var.highly_variable

In [ ]:
scv.tl.recover_dynamics(adata, n_jobs=8)

In [ ]:
scv.tl.velocity(adata, mode="dynamical")
scv.tl.velocity_graph(adata)

In [ ]:
adata.obs

In [ ]:
adata

In [ ]:
scv.pl.velocity_embedding(adata, basis='umap', color='seurat_clusters', dpi=300)

In [ ]:
scv.pl.velocity_embedding_grid(adata, basis='umap', color='seurat_clusters', dpi=300, legend_loc='best')

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis="umap", color='seurat_clusters',
    title="integration both direction", smooth=0.8, dpi=300,
    legend_loc='right', legend_fontsize=12,
    min_mass=4 #5 is the largest velocities (defaults to 1)
)

In [ ]:
scv.pl.velocity_embedding_stream(
    adata, basis="umap", color='Sample',
    title="integration both direction", smooth=0.8, dpi=300,
    legend_loc='right', legend_fontsize=12,
    min_mass=4 #5 is the largest velocities (defaults to 1)
)

In [ ]:
scv.pl.velocity_embedding_stream(
    adata, basis="umap", color='seurat_clusters',
    title="integration forward direction", smooth=0.8, dpi=300,
    legend_loc='right', legend_fontsize=12,
    min_mass=4, #5 is the largest velocities (defaults to 1)
    integration_direction = 'forward'
) # min_mass 1-5 5 means large velocities only.

In [ ]:
scv.pl.velocity_embedding_stream(
    adata, basis="umap", color='seurat_clusters',
    title="integration backward direction", smooth=0.8, dpi=300,
    legend_loc='right', legend_fontsize=12,
    min_mass=4, #5 is the largest velocities (defaults to 1)
    integration_direction = 'backward'
) # min_mass 1-5 5 means large velocities only.

In [ ]:
adata.write('../../cell_rank/datasets/mudt_aldata_merged_scv_dyn.h5ad')

In [ ]:
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')  # for beautified visualization

In [ ]:
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index
scv.pl.scatter(adata,color='seurat_clusters',basis=top_genes[:15], ncols=5, frameon=False, dpi=120)

In [ ]:
# top genes in each clusters
scv.tl.rank_dynamical_genes(adata, n_genes=100, groupby='seurat_clusters') #n_genes : int, optional (default: 100)
#This ranks genes by their likelihood obtained from the dynamical model grouped by clusters specified in groupby.
df = scv.get_df(adata, 'rank_dynamical_genes/names')
df.head(5)

In [ ]:
adata

In [ ]:
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80)

In [ ]:
adata

In [ ]:
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index[:300]
scv.pl.heatmap(adata, var_names=top_genes, sortby='latent_time', 
col_color='seurat_clusters', n_convolve=100)

In [ ]:
# scv.tl.score_genes_cell_cycle(adata) 
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')  # for beautified visualization
scv.pl.scatter(adata, color_gradients=['S.Score', 'G2M.Score'], smooth=True, perc=[5,95], dpi=100)

In [ ]:
scv.tl.velocity_confidence(adata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(adata, c=keys, cmap='coolwarm', perc=[5, 95])

In [ ]:
del df
df = adata.obs.groupby('seurat_clusters')[keys].mean().T
df.style.background_gradient(cmap='coolwarm', axis=1)

In [ ]:
del df

In [ ]:
df = adata.var
df = df[(df['fit_likelihood'] > .1) & df['velocity_genes'] == True]

kwargs = dict(xscale='log', fontsize=16)
with scv.GridSpec(ncols=3) as pl:
    pl.hist(df['fit_alpha'], xlabel='transcription rate', **kwargs)
    pl.hist(df['fit_beta'] * df['fit_scaling'], xlabel='splicing rate', xticks=[.1, .4, 1], **kwargs)
    pl.hist(df['fit_gamma'], xlabel='degradation rate', xticks=[.1, .4, 1], **kwargs)

scv.get_df(adata, 'fit*', dropna=True).head()

In [ ]:
df.to_csv('../../cell_rank/datasets/df_dynamical_kinetic_rate.csv', index=True,
index_label='Index')

In [ ]:
df_reload = pd.read_csv('../../cell_rank/datasets/df_dynamical_kinetic_rate.csv',
index_col=0)  

In [ ]:
del df, df_reload, top_genes

In [ ]:
scv.tl.rank_dynamical_genes(adata, groupby='seurat_clusters')
df = scv.get_df(adata, 'rank_dynamical_genes/names')
df.head(5)

In [ ]:
df.to_csv('../../cell_rank/datasets/df_cluster_specific_rank_dynamical_genes.csv', index=True,
index_label='Index')

In [ ]:
df_reload = pd.read_csv('../../cell_rank/datasets/df_cluster_specific_rank_dynamical_genes.csv',
index_col=0) 

In [ ]:
for cluster in ['0', '1', '2', '3','4','5', '6', '7']:
    scv.pl.scatter(adata, df[cluster][:5], ylabel=cluster, frameon=False, color='seurat_clusters')


In [ ]:
for cluster in ['0', '1', '2', '3','4','5', '6', '7']:
    scv.pl.scatter(adata, df[cluster][:5], ylabel=cluster, frameon=False, 
    color='Sample')


In [ ]:
adata.write('../../cell_rank/datasets/mudt_aldata_merged_scv_dyn_v2.h5ad')

In [ ]:
scv.utils.show_proportions(adata)

In [ ]:
from cellrank.kernels import VelocityKernel
vk = VelocityKernel(adata)

In [ ]:
vk.compute_transition_matrix()

In [ ]:
print(vk)

In [ ]:
from cellrank.kernels import ConnectivityKernel

ck = ConnectivityKernel(adata).compute_transition_matrix()

In [ ]:
combined_kernel = 0.8 * vk + 0.2 * ck

In [ ]:
print(combined_kernel)

In [ ]:
from cellrank.estimators import GPCCA

g = GPCCA(combined_kernel)
print(g)

In [ ]:
g.compute_schur(n_components=20)
g.plot_spectrum()

In [ ]:
adata.obs['seurat_clusters']=adata.obs['seurat_clusters'].astype('category')

In [ ]:
g.compute_macrostates(cluster_key='seurat_clusters')
# n_states=int, if none, use the eigengap heuristic
g.plot_macrostates()

In [ ]:
g.compute_macrostates(n_states=2, cluster_key='seurat_clusters')
g.plot_macrostates()

In [ ]:
g.compute_macrostates(n_states=8, cluster_key='seurat_clusters')
g.plot_macrostates()

In [ ]:
g.plot_macrostates(states=['0', '1_1', '1_2'], same_plot=False)

In [ ]:
g.plot_macrostates(states=['2_1', '2_2', '2_3'], same_plot=False)

In [ ]:
g.plot_macrostates(states=['4', '5'], same_plot=False)

In [ ]:
g.compute_terminal_states() # automatic way--stability criterion

In [ ]:
g.set_terminal_states_from_macrostates(names=['0', '4', '5'])
# LW note: I can keep the terminal_states=["4"] identified in the g.compute_terminal_states()step,
# rather than set more than one in this step (potentially making things harder to interpret).

In [ ]:
g._compute_initial_states()

In [ ]:
g.plot_macrostates(states=['1_1'], color="seurat_clusters")

In [ ]:
adata.obs

In [ ]:
adata.obs['initial_states']

In [ ]:
#g.compute_absorption_probabilities(keys=['0'])
g.compute_absorption_probabilities() #use all terminal states ['0','4','5']

In [ ]:
g.plot_absorption_probabilities(same_plot=False)

In [ ]:
g.plot_absorption_probabilities(same_plot=True)

In [ ]:
zero_drivers = g.compute_lineage_drivers(lineages="0", return_drivers=True)
zero_drivers.sort_values(by="0_corr", ascending=False)

In [ ]:
zero_drivers.to_csv('../sc_cellrank/cell_rank/df_zero_drivers.csv', 
index=True, index_label='index')

In [ ]:
g.plot_lineage_drivers("0", n_genes=5, ncols=3, dpi=300, 
save='../cell_rank/zero_terminal_state_drivers.pdf')
# saved

In [ ]:
cl4_drivers = g.compute_lineage_drivers(lineages="4", return_drivers=True)
cl4_drivers.sort_values(by="4_corr", ascending=False)

In [ ]:
cl4_drivers.to_csv('../sc_cellrank/cell_rank/df_cluster4_drivers.csv', 
index=True, index_label='index')

In [ ]:
g.plot_lineage_drivers("4", n_genes=5, ncols=3, dpi=300, 
save='../cell_rank/cluster4_terminal_state_drivers.pdf')
# saved

In [ ]:
cl5_drivers = g.compute_lineage_drivers(lineages="5", return_drivers=True)
cl5_drivers.sort_values(by="5_corr", ascending=False)

In [ ]:
cl5_drivers.to_csv('../sc_cellrank/cell_rank/df_cluster5_drivers.csv', 
index=True, index_label='index')

In [ ]:
g.plot_lineage_drivers("5", n_genes=5, ncols=3, dpi=100)
#g.plot_lineage_drivers("4", n_genes=5, ncols=3, dpi=300, 
#save='../cell_rank/cluster4_terminal_state_drivers.pdf') # saved!

In [ ]:
adata.write('../../cell_rank/datasets/mudt_aldata_merged_scv_dyn_cr.h5ad')

In [ ]:
g.coarse_initial_distribution()

In [ ]:
# compue DPT, starting from CellRank defined root cell
root_idx = np.where(adata.obs["initial_states"] == "1_1")[0][0]
adata.uns["iroot"] = root_idx
sc.tl.dpt(adata)

In [ ]:
scv.pl.scatter(
    adata,
    color=["seurat_clusters", root_idx],
    fontsize=16,
    cmap="viridis",
    perc=[2, 98],
    colorbar=True,
    rescale_color=[0, 1],
    title=["clusters", "root cell"],
    dpi=900
)

In [ ]:
scv.pl.scatter(
    adata,
    color=[ "latent_time", "dpt_pseudotime"],
    fontsize=16,
    cmap="viridis",
    perc=[2, 98],
    colorbar=True,
    rescale_color=[0, 1],
    title=["latent time", "dpt pseudotime"],
    dpi=900
)

In [ ]:
combined_kernel = 0.8 * vk + 0.2 * ck
print(combined_kernel)

In [ ]:
k = VelocityKernel(adata).compute_transition_matrix()
k

In [ ]:
vk

In [ ]:
ck

In [ ]:
combined_kernel.plot_random_walks(
    100,
    start_ixs={"seurat_clusters": "2"},
    max_iter=100,
    show_progress_bar=False,
    ixs_legend_loc="best",
    seed=42,
)

In [ ]:
combined_kernel.plot_random_walks(
    n_sims=300,
    max_iter=200,
    start_ixs={"seurat_clusters": "0"},
    show_progress_bar=False,
    ixs_legend_loc="best",
    cmap="gnuplot",
    seed=42,
    linealpha=0.5,
    dpi=150 #cmap="gnuplot"
)

In [ ]:
combined_kernel.plot_random_walks(
    100,
    start_ixs={"seurat_clusters": "0"},
    stop_ixs={"seurat_clusters": ["4", "5"]},
    max_iter=100,
    successive_hits=5,
    show_progress_bar=False,
    cmap="viridis",
    seed=42,
    ixs_legend_loc="best"
)

In [ ]:
cr.pl.circular_projection(adata, keys="seurat_clusters", legend_loc="right")